In [1]:
# Copyright 2020 Fabian Hofmann
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# IFTM for Video Generation

This notebook serves as an introduction (not an evaluation) of IFTM using video generation techniques.

Identity Function and Threshold Model (IFTM) by Schmidt et al. is an anomaly detection approach that utilizes identity functions and forecasting mechanisms to detect anomalies, by either reconstructing the current datapoint or forecasting a future one. Then the predicted value is compared to the actual data by a reconstruction or prediction error function. To detect anomalies, a threshold is applied to the result -- if it exceeds the threshold it is anomaly, else it is not.

We provide IFTM for video generation as a wrapper around *any* kind of Keras model; the code of the framework class can be found [here](https://gitlab.tubit.tu-berlin.de/sulandir/Thesis/blob/master/src/iftm/iftm.py). Threshold for our IFTM implementation is exclusively trained by using cumulative aggregation (CA), simply because the Keras model is assumed to be trained in batch-mode/offline as well.

##### Sources
- [IFTM by Schmidt et al.](https://www.researchgate.net/publication/327484223_IFTM_-_Unsupervised_Anomaly_Detection_for_Virtualized_Network_Function_Services)
- [Our implementation of IFTM](https://gitlab.tubit.tu-berlin.de/sulandir/Thesis/blob/master/src/iftm/iftm.py)

## Setup

In [2]:
import tensorflow as tf

tf.__version__

'2.5.0-dev20201111'

In [3]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [4]:
import numpy as np
import os
import time

from src.io_.batch_generator import UniformBatchGenerator, BatchLoader
from src.models.vgan import *
from src.models.iftm import IFTM

In [5]:
base_input_path = "../../data/upCam/preprocessed_128_64/"
base_output_path = "../../output/models/iftm/"

### Init batch generator for training and testing

The 5th day is used for "testing" (not really testing, but just seeing how well the model classifies unknown normal data).

In [6]:
days_training = ["00", "01", "02", "03"]
day_paths_training = [base_input_path + day + "/" for day in days_training]
training_generator = UniformBatchGenerator(day_paths=day_paths_training, batch_size=64, sample_size=8, subsample_size=7)

In [7]:
days_testing = ["04"]
day_paths_testing = [base_input_path + day + "/" for day in days_testing]
testing_generator = UniformBatchGenerator(day_paths=day_paths_testing, batch_size=64, sample_size=8, subsample_size=7)

## C-VGAN Models

C-VGAN models are built and trained just like in [vgan_conditional_3d_2.ipynb](https://gitlab.tubit.tu-berlin.de/sulandir/Thesis/blob/master/src/models/vgan_conditional_3d_2.ipynb).

### Model init

In [8]:
def make_generator_model() -> keras.Model:
    inputs: tf.Tensor = keras.Input(shape=(7, 64, 128, 3))

    e_3d = make_encoder_foreground_stream(inputs)
    f, m = make_conditional_foreground_stream(e_3d)

    e_2d = make_encoder_background_stream(inputs)
    b = make_conditional_background_stream(e_2d)

    outputs = make_generator_stream_combiner(f, m, b)

    return keras.Model(inputs=inputs, outputs=outputs, name="c_vgan_generator")

In [9]:
c_vgan_generator = make_generator_model()

In [10]:
c_vgan_discriminator = make_discriminator_model()

### Loss and optimizers init

In [11]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
mean_abs_err = tf.keras.losses.MeanAbsoluteError()

In [12]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

In [13]:
LAMBDA = 10

def generator_loss(fake_output, input_videos, generated_videos):
    fake_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    reconstruction_loss = mean_abs_err(input_videos, generated_videos[:,:7]) * LAMBDA
    return fake_loss + reconstruction_loss

In [14]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)

## Save checkpoints

In [15]:
checkpoint_dir = base_output_path + "training_checkpoints/"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=c_vgan_generator,
                                 discriminator=c_vgan_discriminator)

### Define the training loop

In [16]:
@tf.function
def train_step(batch):
    input_videos, real_videos = batch

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_videos = c_vgan_generator(input_videos, training=True)

        real_output = c_vgan_discriminator(real_videos, training=True)
        fake_output = c_vgan_discriminator(generated_videos, training=True)

        gen_loss = generator_loss(fake_output, input_videos, generated_videos)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, c_vgan_generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, c_vgan_discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, c_vgan_generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, c_vgan_discriminator.trainable_variables))

    return gen_loss, disc_loss

In [17]:

def train(epochs):
    # Load latest checkpoint if available
    latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    checkpoint.restore(latest_checkpoint)
    if latest_checkpoint:
        print("Restored from {}".format(latest_checkpoint))
        return
    else:
        print("Initializing from scratch.")

    for epoch in range(epochs):
        # Init batch loader for this epoch
        batch_loader = BatchLoader(training_generator, max_queue_size=32, no_workers=4)

        start = time.time()

        # Train models over the entire batch each epoch
        for _ in range(len(batch_loader)):
            train_step(batch_loader.get_batch())

        # Save the model every 2 epochs
        if (epoch + 1) % 2 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

        print('Time for epoch {} is {} sec'.format(epoch + 1, time.time() - start))

        # Cleanup
        batch_loader.shutdown_workers()

## Train the model

Train C-VGAN for a single epoch to get some results for IFTM and not just random prediction. Actual training is done during evaluation in another notebook.

In [18]:
train(1)

Initializing from scratch.
Time for epoch 1 is 985.5255591869354 sec


## IFTM

### Define the error function

In the original IFTM paper L2 was utilized to compute the reconstruction error. For evaluation purposes we chose to stick to L1 however, because that term was minimized during training (as in the original VGAN paper). But, because only the forecast frame matters, we exclude the (seven) reconstructed frames from the error and focus on the 8th frame of the generated video (and its actual counterpart).

In [19]:
reconstruction_err = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.NONE)

def error_function(predicted: tf.Tensor, actual: tf.Tensor) -> tf.Tensor:
    predicted = tf.reshape(predicted[:,-1], shape=[predicted.shape[0],-1])
    actual = tf.reshape(actual[:,-1], shape=[actual.shape[0],-1])
    return reconstruction_err(predicted, actual)

### Init and train IFTM

In [20]:
iftm = IFTM(c_vgan_generator, error_function)

The training data that was used to train the C-VGAN model is now rerun on the model to compute the training error to train the threshold model of IFTM as well.

In [21]:
start_tm_training = time.time()
iftm.train_threshold(training_generator, max_queue_size=128, no_workers=4)

print('Time for TM training is {} sec'.format(time.time() - start_tm_training))

Time for TM training is 301.5885262489319 sec


Print out the threshold:

In [22]:
print(iftm.threshold)

0.056033224


### Make predictions

In [23]:
start_iftm_testing = time.time()

testing_batches = BatchLoader(testing_generator, max_queue_size=32, no_workers=4)
predictions = []
for _ in range(len(testing_batches)):
    b_x, b_y = testing_batches.get_batch()
    _, p = iftm.predict(b_x, b_y)
    predictions.append(p)
testing_batches.shutdown_workers()

print('Time for IFTM testing is {} sec'.format(time.time() - start_iftm_testing))

Time for IFTM testing is 75.47670412063599 sec


Count and output the number of "false" (0, normal) and "true" (1, anomaly) predictions.

In [24]:
predictions = np.array(predictions).flatten()
p_count = np.bincount(predictions)

print(p_count)

[18230 35530]
